<a href="https://colab.research.google.com/github/clitwiniuk/Python_para_Trading/blob/master/vwap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ta
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip install pydrive
!apt-get install -yq sqlite3

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=7aadd0088bc6109f5d0859fca5759e67a6c7b6eac38099be8d82068702bcb7b6
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 58.3 MB/s eta 0:00:00
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.151.0
    Uninstalling google-api-python-client-2.151.0:
      Successfully uninstalled google-api-python-client-2.151.0
Reading package lists...
Building dependency tree...
Reading state information...
Suggested packages:
  sqlite3-doc
The following NEW packages will be installed:
  sqlite3
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 768 kB of archives.
After this operation, 1,873 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu j

In [ ]:
# Celda 1: Importaciones
import pandas as pd
import numpy as np
import ta
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
from enum import Enum
from typing import Dict, List
from dataclasses import dataclass
from datetime import datetime
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
%matplotlib inline

In [ ]:
# Celda 2: Obtener datos de la base de datos
from google.colab import drive

# Force remount to refresh authentication
drive.mount('/content/drive', force_remount=True)

# Ruta al archivo en Google Drive
file_path = '/content/drive/MyDrive/Data/trendvision_alerts_historico.db'

# Conexion a base de datos
conexion = sqlite3.connect(file_path)

query = """
SELECT ticker, date, open, high, low, close, volume
FROM ohlc
ORDER BY date, ticker
"""

data = pd.read_sql_query(query, conexion)
data['date'] = pd.to_datetime(data['date'])

# Cargar las alertas desde la tabla
alerts_query = "SELECT * FROM alerts"
alerts = pd.read_sql_query(alerts_query, conexion)



print(f"Fecha disponible: {data['date'].unique()[0]}")
print(f"Número de tickers: {data['ticker'].nunique()}")
print(f"Tickers disponibles: {', '.join(data['ticker'].unique())}")


Mounted at /content/drive
Fecha disponible: 2024-10-08 15:30:00
Número de tickers: 231
Tickers disponibles: ACCD, ADTX, AEON, AGFY, AGRI, AKTS, ALBT, ALCE, AMBO, APPS, BJDX, BLMZ, BOF, BPTH, BQ, BURU, CETX, CHNR, CLEU, CNVS, CODX, CTCX, DAO, DBGI, ENGN, EONR, FUTU, GLXG, GNLN, GNS, GRI, GSIW, HOLO, HOVR, IBG, IHRT, INKT, JTAI, KC, KDLY, KXIN, KZR, LASE, LGMK, LUXH, LX, MLEC, MLGO, MYNZ, MYTE, NIO, NMHI, NNOX, NOVA, NXL, OPAD, OTRK, PALT, PBM, PETZ, PEV, POLA, QNRX, RAPT, RHE, RIME, RLX, RNAZ, RR, RVSN, SAR, SAVA, SNAL, SOBR, STTK, SVMH, TIGR, TIVC, TNXP, TURB, TVGN, VCNX, VERB, VINC, VMAR, VRCA, VVOS, WETH, WW, XPON, XXII, EDBL, ICCT, AMST, SUGP, ALTM, APLD, ATNF, BLUE, BMEA, BNZI, BSFC, CLSD, CUE, DHX, DUO, EFSH, EJH, ELAB, ETWO, FFIE, GWAV, HAO, IKT, INHD, IVP, IZEA, LGVN, MNTS, MULN, MYND, NISN, NUWE, NVAX, OCGN, ONDS, PDYN, SAVE, SISI, SKYX, SMXT, SONN, SRFM, STKL, TRNR, VCIG, VIR, VRAX, VTRS, WHLR, WINT, WSR, XAIR, ZJYL, ZK, MPU, AILE, FTRE, SGN, SPPL, NRSN, SOAR, INAB, AEHR, ALLK

In [ ]:
# Celda 3: Clases base
class OrderType(Enum):
    MARKET = "MARKET"
    LIMIT = "LIMIT"
    STOP = "STOP"

@dataclass
class Order:
    symbol: str
    order_type: OrderType
    size: int
    price: float = 0.0
    stop_price: float = 0.0
    timestamp: datetime = None


In [ ]:
# Celda 4: Clase Estrategia Base
class BaseStrategy:
    def __init__(self, data: pd.DataFrame, params: Dict = None):
        self.data = data.copy()
        self.position = {}
        self.trades = []
        self.orders = []
        self.entry_prices = {}  # Diccionario para guardar precios de entrada


        self.params = {
            'stop_loss_pct': 0.02,
            'take_profit_pct': 0.03
        }
        if params:
            self.params.update(params)

    def place_order(self, order: Order):
        self.orders.append({
            'timestamp': order.timestamp,
            'ticker': order.symbol,
            'type': order.order_type.value,
            'size': order.size,
            'price': order.price
        })

        # Actualizar posición
        if order.symbol not in self.position:
            self.position[order.symbol] = 0
        self.position[order.symbol] += order.size

        # Registrar trade
        self.trades.append({
            'timestamp': order.timestamp,
            'ticker': order.symbol,
            'price': order.price,
            'size': order.size,
            'position': self.position[order.symbol]
        })

In [ ]:
# Celda 5: Estrategia Específica
class MiEstrategia(BaseStrategy):
    def __init__(self, data: pd.DataFrame, params: Dict = None):
        self.default_params = {
            'cantidad_compra': 1,  # número de acciones a comprar/vender
            'stop_loss_pct': 0.02  # stop loss opcional
        }

        if params:
            self.default_params.update(params)

        super().__init__(data, self.default_params)

    def calculate_indicators(self):
        """Calcula el VWAP para cada ticker"""
        for ticker in self.data['ticker'].unique():
            mask = self.data['ticker'] == ticker
            df_ticker = self.data[mask]

            # Calcular VWAP
            cumulative_tp_v = (df_ticker['close'] * df_ticker['volume']).cumsum()
            cumulative_volume = df_ticker['volume'].cumsum()
            self.data.loc[mask, 'vwap'] = cumulative_tp_v / cumulative_volume

    def run_strategy(self):
        """Ejecuta la estrategia: Compra al abrir mercado y vende según VWAP"""
        self.calculate_indicators()

        for ticker in self.data['ticker'].unique():
            ticker_data = self.data[self.data['ticker'] == ticker].copy()

            # Compra al abrir el mercado: primer registro
            first_row = ticker_data.iloc[0]
            order = Order(
                symbol=ticker,
                order_type=OrderType.MARKET,
                size=self.params['cantidad_compra'],
                price=first_row['open'],  # Precio de apertura
                timestamp=first_row['date']
            )
            self.place_order(order)
            print(f"COMPRA INICIAL {ticker}: Precio {first_row['open']:.2f}")

            # Iterar sobre el resto de los registros
            for i in range(1, len(ticker_data)):
                current_row = ticker_data.iloc[i]
                current_position = self.position.get(ticker, 0)

                # Venta: si el precio cae por debajo del VWAP
                if (current_position > 0 and
                    current_row['close'] < current_row['vwap']):
                    order = Order(
                        symbol=ticker,
                        order_type=OrderType.MARKET,
                        size=-current_position,  # Vender toda la posición
                        price=current_row['close'],
                        timestamp=current_row['date']
                    )
                    self.place_order(order)
                    print(f"VENTA {ticker}: Precio {current_row['close']:.2f} < VWAP {current_row['vwap']:.2f}")

                # Stop Loss: si el precio cae un 8% por debajo del precio de entrada
                elif (current_position > 0 and
                      current_row['close'] < self.entry_prices.get(ticker, float('inf')) * 0.92):
                    order = Order(
                        symbol=ticker,
                        order_type=OrderType.MARKET,
                        size=-current_position,
                        price=current_row['close'],
                        timestamp=current_row['date']
                    )
                    self.place_order(order)
                    print(f"STOP LOSS {ticker}: Precio {current_row['close']:.2f} (8% debajo de entrada)")



    def print_results(self):
        """Imprime los resultados de la estrategia"""
        if not self.trades:
            print("No se realizaron operaciones")
            return

        df_trades = pd.DataFrame(self.trades)

        print("\nResumen de operaciones:")
        print(f"Número total de operaciones: {len(df_trades)}")

        # Calcular P&L
        df_trades['pnl'] = df_trades['size'] * df_trades['price'] * -1
        df_trades['cumulative_pnl'] = df_trades['pnl'].cumsum()

        print(f"P&L final: ${df_trades['cumulative_pnl'].iloc[-1]:.2f}")

        # Visualizar P&L acumulado
        plt.figure(figsize=(12, 6))
        plt.plot(df_trades.index, df_trades['cumulative_pnl'])
        plt.title('P&L Acumulado')
        plt.xlabel('Número de operación')
        plt.ylabel('P&L ($)')
        plt.grid(True)
        plt.show()

In [ ]:
# Celda 6: Procesar alertas y ejecutar estrategia basada en ellas
# Configurar parámetros generales de la estrategia
params = {
    'cantidad_compra': 100,
    'stop_loss_pct': 0.02
}

# Cargar las alertas desde la tabla
alerts_query = "SELECT * FROM alerts"
alerts = pd.read_sql_query(alerts_query, conexion)

results = []

# Procesar cada alerta
for _, alert in alerts.iterrows():
    ticker = alert['ticker']
    alert_date = pd.to_datetime(alert['date'])
    entry_price = alert['price']  # Precio en el momento de la alerta

    # Filtrar datos OHLC del ticker desde el momento de la alerta
    ohlc_query = f"""
        SELECT * FROM ohlc
        WHERE ticker = '{ticker}'
          AND date >= '{alert_date}'
        ORDER BY date
    """
    ohlc_data = pd.read_sql_query(ohlc_query, conexion)

    if ohlc_data.empty:
        print(f"No se encontraron datos para {ticker} después de la alerta en {alert_date}.")
        continue

    # Crear estrategia y ejecutarla
    estrategia = MiEstrategia(ohlc_data, params)
    estrategia.run_strategy()

    # Extraer resultados de trades para este ticker
    trades = pd.DataFrame(estrategia.trades)
    if not trades.empty:
        # Agregar datos adicionales de la alerta para análisis
        trades['alert_date'] = alert_date
        trades['entry_price'] = entry_price
        trades['ticker'] = ticker
        results.append(trades)

# Combinar todos los resultados de las alertas procesadas
if results:
    results_df = pd.concat(results, ignore_index=True)

    print("\nResumen de operaciones:")
    print(results_df)

    # Calcular y visualizar el P&L acumulado
    results_df['pnl'] = results_df['size'] * results_df['price'] * -1
    results_df['cumulative_pnl'] = results_df['pnl'].cumsum()

    plt.figure(figsize=(12, 6))
    plt.plot(results_df.index, results_df['cumulative_pnl'])
    plt.title('P&L Acumulado')
    plt.xlabel('Número de operación')
    plt.ylabel('P&L ($)')
    plt.grid(True)
    plt.show()
else:
    print("No se ejecutaron operaciones.")


conexion.close()


COMPRA INICIAL FUTU: Precio 104.90
VENTA FUTU: Precio 105.76 < VWAP 105.79
COMPRA INICIAL NIO: Precio 6.26
VENTA NIO: Precio 6.17 < VWAP 6.19
COMPRA INICIAL KC: Precio 3.63
VENTA KC: Precio 3.60 < VWAP 3.61
COMPRA INICIAL BURU: Precio 0.62
STOP LOSS BURU: Precio 0.62 (8% debajo de entrada)
COMPRA INICIAL RLX: Precio 1.74
STOP LOSS RLX: Precio 1.78 (8% debajo de entrada)
COMPRA INICIAL OTRK: Precio 3.04
STOP LOSS OTRK: Precio 3.11 (8% debajo de entrada)
COMPRA INICIAL SOBR: Precio 8.16
STOP LOSS SOBR: Precio 9.32 (8% debajo de entrada)
COMPRA INICIAL RNAZ: Precio 0.50
STOP LOSS RNAZ: Precio 0.51 (8% debajo de entrada)
COMPRA INICIAL CHNR: Precio 0.92
STOP LOSS CHNR: Precio 0.91 (8% debajo de entrada)
COMPRA INICIAL XXII: Precio 0.18
STOP LOSS XXII: Precio 0.18 (8% debajo de entrada)
COMPRA INICIAL CETX: Precio 1.03
VENTA CETX: Precio 1.00 < VWAP 1.02
COMPRA INICIAL RVSN: Precio 0.54
STOP LOSS RVSN: Precio 0.54 (8% debajo de entrada)
COMPRA INICIAL VINC: Precio 0.65
STOP LOSS VINC: Preci

ZeroDivisionError: float division by zero

In [ ]:
# Celda 7: Analizar resultados
trades_df = pd.DataFrame(estrategia.trades)
if len(trades_df) > 0:
    print("\nResumen de operaciones:")
    print(f"Total de operaciones: {len(trades_df)}")
    print("\nOperaciones por ticker:")
    print(trades_df.groupby('ticker').size())

    # Calcular P&L por ticker
    trades_df['pnl'] = trades_df.apply(lambda x: x['size'] * x['price'] * -1, axis=1)
    trades_df['cumulative_pnl'] = trades_df.groupby('ticker')['pnl'].cumsum()

    print("\nP&L por ticker:")
    print(trades_df.groupby('ticker')['pnl'].sum())

    # Visualizar P&L acumulado por ticker
    plt.figure(figsize=(15, 7))
    for ticker in trades_df['ticker'].unique():
        ticker_data = trades_df[trades_df['ticker'] == ticker]
        plt.plot(ticker_data.index, ticker_data['cumulative_pnl'],
                 label=ticker, alpha=0.7)
    plt.title('P&L Acumulado por Ticker')
    plt.legend()
    plt.show()
else:
    print("No se ejecutaron operaciones")
